In [13]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import random

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import re

In [15]:
#GOOOOOD IT WORKS!!!

options = Options()
options.add_argument("user-data-dir=C:/Users/znay/AppData/Local/Google/Chrome/User Data")
options.add_argument(r'--profile-directory=Profile 6') #e.g. Profile 3
options.add_experimental_option("useAutomationExtension", False)
options.add_experimental_option("excludeSwitches",["enable logging"])
options.add_experimental_option("excludeSwitches", ["enable automation"])
options.add_argument("start-maximized")

driver = webdriver.Chrome(service = Service(executable_path=r"C:\Users\znay\Downloads\chromedriver.exe"), options = options)

#driver.get(r"https://medium.com/swlh/five-simple-ways-to-drastically-boost-your-energy-26f5ec19ad81")

#m = driver.find_element(By.XPATH, "/html/body").text

#soup = BeautifulSoup(m, 'html.parser')

In [16]:

options = Options()
options.add_argument("user-data-dir=C:/Users/znay/AppData/Local/Google/Chrome/User Data")
options.add_argument(r'--profile-directory=Profile 6') #e.g. Profile 3
options.add_experimental_option("useAutomationExtension", False)
options.add_experimental_option("excludeSwitches",["enable logging"])
options.add_experimental_option("excludeSwitches", ["enable automation"])
options.add_argument("start-maximized")

driver = webdriver.Chrome(service = Service(executable_path=r"C:\Users\znay\Downloads\chromedriver.exe"), options = options)

import pandas as pd
from bs4 import BeautifulSoup
import requests


def scraper(ogurl, year,csv_save_url):

    stories_data = []

    for month in range(1, 13):
        if month in [1, 3, 5, 7, 8, 10, 12]:
            n_days = 31
        elif month in [4, 6, 9, 11]:
            n_days = 30
        else:
            n_days = 28

        for day in range(1, n_days + 1):

            month, day = str(month), str(day)

            if len(month) == 1:
                month = f'0{month}'
            if len(day) == 1:
                day = f'0{day}'

            date = f'{month}/{day}/{year}'
            url = f'{ogurl}/archive/{year}/{month}/{day}'
            print(url)
            page = requests.get(url)
            soup = BeautifulSoup(page.text, 'html.parser')

            stories = soup.find_all('div', class_='streamItem streamItem--postPreview js-streamItem')
            j=0
            for raw_story in stories:
                j=0
                try:
                    each_story = []

                    match = re.findall(r'data-action-source="preview-listing" data-action-value="(.*?=collection_archive)', str(raw_story))
                    driver.get(match)

                    text = driver.find_element(By.XPATH, "/html/body").text

                    story = BeautifulSoup(text, 'html.parser')

                    author_box = story.find('div', class_='postMetaInline u-floatLeft u-sm-maxWidthFullWidth')
                    author_url = author_box.find('a')['href']

                    try:
                        reading_time = author_box.find('span', class_='readingTime')['title']
                    except:
                        continue

                    title = story.find('h3').text if story.find('h3') else '-'

                    subtitle = story.find('h4').text if story.find('h4') else '-'

                    if story.find('button', class_='button button--chromeless u-baseColor--buttonNormal'
                                               ' js-multirecommendCountButton u-disablePointerEvents'):

                        claps = story.find('button', class_='button button--chromeless u-baseColor--buttonNormal'
                                                        ' js-multirecommendCountButton u-disablePointerEvents').text
                    else:
                        claps = 0

                    if story.find('a', class_='button button--chromeless u-baseColor--buttonNormal'):
                        responses = story.find('a', class_='button button--chromeless u-baseColor--buttonNormal').text
                    else:
                        responses = '0 responses'

                    story_url = story.find('a', class_='button button--smaller button--chromeless u-baseColor--buttonNormal')[
                    'href']

                    reading_time = reading_time.split()[0]
                    responses = responses.split()[0]

                    story_page = requests.get(story_url)
                    story_soup = BeautifulSoup(story_page.text, 'html.parser')

                    sections = story_soup.find_all('section')
                    story_paragraphs = []
                    section_titles = []
                    for section in sections:
                        paragraphs = section.find_all('p')
                        for paragraph in paragraphs:
                            story_paragraphs.append(paragraph.text)

                        subs = section.find_all('h1')
                        for sub in subs:
                            section_titles.append(sub.text)

                    number_sections = len(section_titles)
                    number_paragraphs = len(story_paragraphs)

                    each_story.append(date)
                    each_story.append(title)
                    each_story.append(subtitle)
                    each_story.append(claps)
                    each_story.append(responses)
                    each_story.append(author_url)
                    each_story.append(story_url)
                    each_story.append(reading_time)
                    each_story.append(number_sections)
                    each_story.append(section_titles)
                    each_story.append(number_paragraphs)
                    each_story.append(story_paragraphs)

                    stories_data.append(each_story)
                except:
                    j+=1
                    continue


            print(f'{len(stories)-j} stories scraped on {date}.')

    columns = ['date', 'title', 'subtitle', 'claps', 'responses', 'author_url', 'story_url',
               'reading_time (mins)', 'number_sections', 'section_titles', 'number_paragraphs', 'paragraphs']

    df = pd.DataFrame(stories_data, columns=columns)
    df.to_csv(csv_save_url, sep='\t', index=False)
    return df

In [27]:
page = requests.get(r"https://towardsdatascience.com//archive/2023/01/01")
soup = BeautifulSoup(page.text, 'html.parser')
stories  =  soup.find_all('div', class_='streamItem streamItem--postPreview js-streamItem')

In [29]:
stories[0].find('div', class_='postMetaInline u-floatLeft u-sm-maxWidthFullWidth')

<div class="postMetaInline u-floatLeft u-sm-maxWidthFullWidth"><div class="u-flexCenter"><div class="postMetaInline-avatar u-flex0"><a class="link u-baseColor--link avatar" data-action="show-user-card" data-action-type="hover" data-action-value="4d4f8ddd1e68" data-collection-slug="towards-data-science" data-user-id="4d4f8ddd1e68" dir="auto" href="https://towardsdatascience.com/@mgalkin"><img alt="Go to the profile of Michael Galkin" class="avatar-image u-size36x36 u-xs-size32x32" src="https://cdn-images-1.medium.com/fit/c/72/72/2*R6303tLavDAf6jJAsMlaJQ.jpeg"/></a></div><div class="postMetaInline postMetaInline-authorLockup ui-captionStrong u-flex1 u-noWrapWithEllipsis"><a class="ds-link ds-link--styleSubtle link link--darken link--accent u-accentColor--textNormal u-accentColor--textDarken" data-action="show-user-card" data-action-source="collection_archive---------0-----------------------" data-action-type="hover" data-action-value="4d4f8ddd1e68" data-collection-slug="towards-data-scie

In [24]:
match = re.findall(r'data-action-source="preview-listing" data-action-value="(.*?=collection_archive)', str(stories[0]))
options = Options()
options.add_argument("user-data-dir=C:/Users/znay/AppData/Local/Google/Chrome/User Data")
options.add_argument(r'--profile-directory=Profile 9') #e.g. Profile 3
options.add_experimental_option("useAutomationExtension", False)
options.add_experimental_option("excludeSwitches",["enable logging"])
options.add_experimental_option("excludeSwitches", ["enable automation"])
options.add_argument("start-maximized")

driver = webdriver.Chrome(service = Service(executable_path=r"C:\Users\znay\Downloads\chromedriver.exe"), options = options)

driver.get(match[0])
text = driver.find_element(By.XPATH, "/html/body").text

story = BeautifulSoup(text, 'html.parser')

author_box = story.find('div', class_='postMetaInline u-floatLeft u-sm-maxWidthFullWidth')

In [26]:
story

Write
STATE OF THE ART DIGEST
Graph ML in 2023: The State of Affairs
Hot trends and major advancements
Michael Galkin
·
Follow
Published in
Towards Data Science
·
26 min read
·
Jan 1, 2023
927
3
2022 comes to an end and it is about time to sit down and reflect upon the achievements made in Graph ML as well as to hypothesize about possible breakthroughs in 2023. Tune in 🎄☕
Background image generated by DALL-E 2, text added by Author.
The article is written together with Hongyu Ren (Stanford University), Zhaocheng Zhu (Mila &amp; University of Montreal). We thank Christopher Morris and Johannes Brandstetter for the feedback and helping with the Theory and PDE sections, respectively. Follow Michael, Hongyu, Zhaocheng, Christopher, and Johannes here on Medium and Twitter for more graph ml-related discussions.
Table of Contents:
Generative Models: Denoising Diffusion for Molecules and Proteins
DFTs, ML Force Fields, Materials, and Weather Simulations
Geometry &amp; Topology &amp; PDEs
Graph

In [10]:
match[0]

'https://towardsdatascience.com/graph-ml-in-2023-the-state-of-affairs-1ba920cb9232?source=collection_archive'

In [ ]:
swlh_df_2023 = scraper(r"https://medium.com/swlh/",2023,'swlh_df_2023.csv')
swlh_df_2022 = scraper(r"https://medium.com/swlh/",2022,'swlh_df_2022.csv')
swlh_df_2021 = scraper(r"https://medium.com/swlh/",2021,'swlh_df_2021.csv')

In [15]:
towards_df_2023 = scraper(r"https://towardsdatascience.com/", 2023,'towards_df_2023.csv')
towards_df_2022 = scraper(r"https://towardsdatascience.com/", 2022,'towards_df_2022.csv')
towards_df_2021 = scraper(r"https://towardsdatascience.com/", 2021,'towards_df_2021.csv')


https://towardsdatascience.com//archive/2023/01/01


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=122.0.6261.95)
Stacktrace:
	GetHandleVerifier [0x00007FF61986AD22+56930]
	(No symbol) [0x00007FF6197DF622]
	(No symbol) [0x00007FF6196942E5]
	(No symbol) [0x00007FF619671D4C]
	(No symbol) [0x00007FF6197023F7]
	(No symbol) [0x00007FF619717891]
	(No symbol) [0x00007FF6196FBA43]
	(No symbol) [0x00007FF6196CD438]
	(No symbol) [0x00007FF6196CE4D1]
	GetHandleVerifier [0x00007FF619BE6AAD+3709933]
	GetHandleVerifier [0x00007FF619C3FFED+4075821]
	GetHandleVerifier [0x00007FF619C3817F+4043455]
	GetHandleVerifier [0x00007FF619909756+706710]
	(No symbol) [0x00007FF6197EB8FF]
	(No symbol) [0x00007FF6197E6AE4]
	(No symbol) [0x00007FF6197E6C3C]
	(No symbol) [0x00007FF6197D68F4]
	BaseThreadInitThunk [0x00007FFE4243257D+29]
	RtlUserThreadStart [0x00007FFE441EAA58+40]


In [ ]:
import torch
from transformers import pipeline

def is_generated_by_ai(paragraph):
    # Load the text classification pipeline
    text_classifier = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment")

    # Classify the input paragraph
    result = text_classifier(paragraph)

    # You can adjust this threshold based on experimentation
    confidence_threshold = 0.7

    # Check if the label is consistent with AI-generated text
    label = result[0]['label']
    confidence = result[0]['score']
    if label == 'LABEL_1' and confidence >= confidence_threshold:
        return True
    else:
        return False

# Example usage
input_paragraph = "This is an example paragraph."
generated_by_ai = is_generated_by_ai(input_paragraph)

if generated_by_ai:
    print("The paragraph seems to be generated by AI.")
else:
    print("The paragraph seems to be written by a human.")

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("Hello-SimpleAI/chatgpt-detector-roberta")
model = AutoModelForSequenceClassification.from_pretrained("Hello-SimpleAI/chatgpt-detector-roberta")

In [ ]:
from transformers import TextClassificationPipeline
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)
pipe("I have generated some mock data in Python that is obviously not linearly separable. However, there is a clear pattern separating the two classes which gives us an indication that we could create a function that splits the data in a higher dimension. So how about instead of trying to classify this data in two dimensions, we map the data to a three-dimensional space, and try to classify there.")

In [ ]:
pipe("Amelia sat on the porch, sipping her coffee, gazing at the old oak tree in her backyard. Its branches whispered tales of time, and today, they carried a peculiar message. As she listened, a small bird landed on her shoulder, its song harmonizing with the rustling leaves. Suddenly, a letter floated down from the tree, tied with a crimson ribbon. Intrigued, she opened it to find words penned by her late grandmother, a letter lost for decades. Through tears and laughter, the oak tree became the keeper of family stories, connecting generations in a dance of memories under the vast sky.")

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
options = Options()
options.add_argument("user-data-dir=C:/Users/znay/AppData/Local/Google/Chrome/User Data")
options.add_argument(r'--profile-directory=Profile 6') #e.g. Profile 3
options.add_experimental_option("useAutomationExtension", False)
options.add_experimental_option("excludeSwitches",["enable logging"])
options.add_experimental_option("excludeSwitches", ["enable automation"])
options.add_argument("start-maximized")

driver = webdriver.Chrome(service = Service(executable_path=r"C:\Users\znay\Downloads\chromedriver.exe"), options = options)

driver.get(r"https://medium.com/swlh/five-simple-ways-to-drastically-boost-your-energy-26f5ec19ad81")

m = driver.find_element(By.XPATH, "/html/body").text

soup = BeautifulSoup(m, 'html.parser')

In [13]:
soup

Write
Top highlight
Member-only story
Five Simple Ways to Drastically Boost Your Energy
Workers are tired — and they care about their well-being more than ever.
Aytekin Tank
·
Follow
Published in
The Startup
·
6 min read
·
Jan 9, 2023
712
15
Photo by Rachel McDermott on Unsplash
In the wake of multiple crises in the past few years, entrepreneurs are now facing a new one: a human energy crisis. As it turns out, the workplace flexibility afforded by the pandemic came at a cost.
As Kathleen Hogan, Microsoft’s Chief People Officer and EVP, writes, “Since the start of the pandemic, we’ve seen workday span increase more than 13% and after-hours and weekend work are up 28% and 14% respectively.”
The result? Workers around the globe are exhausted. And at the same time, they care more about their health and well-being than ever. In Microsoft’s annual Work Trend Index, a survey of thousands of workers worldwide, 53% of respondents said they’re more likely to prioritize their health and well-bein